In [ ]:

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [ ]:
%pip install openpyxl

In [17]:
file_path = "./Training_data.xlsx"
dataset = pd.read_excel(file_path)
dataset.head()
features = [
    'INCOME', 'SAVINGS', 'DEBT', 'R_SAVINGS_INCOME',
    'R_DEBT_INCOME', 'R_DEBT_SAVINGS'
]

X = dataset[features]

X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Train the XGBoost Model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
model.fit(X_train, X_train.mean(axis=1))  # Train model without a target column

# Save the trained model
joblib.dump(model, 'loan_predictor_model.pkl')
print("Model training completed and saved as 'loan_predictor_model.pkl'.")

# Generate predictions
predictions = model.predict(X_test)

# Save predictions to a new file
output_df = X_test.copy()
output_df["PREDICTED_SAFE_LOAN"] = predictions

output_file = "loan_predictions.xlsx"
output_df.to_excel(output_file, index=False)

print(f"Predictions saved to '{output_file}'.")


Model training completed and saved as 'loan_predictor_model.pkl'.
Predictions saved to 'loan_predictions.xlsx'.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
model = joblib.load("loan_predictor_model.pkl")

In [ ]:
def predict_safe_loan (income,savings,debt,loan_duration,change_Income):
    if change_Income == True:
        new_Income = float(input("Enter new income: "))
        change_in_year = int(input("Enter months after which income changes: "))/12
        adjusted_income = ((income * (loan_duration - change_in_year)) + (change_in_year * new_Income)) / loan_duration
        ratio_savings_income = savings / adjusted_income
        ratio_debt_income = debt / adjusted_income
        ratio_debt_savings = debt / savings
    input_data = pd.DataFrame([{
        "INCOME": adjusted_income,
        "SAVINGS": savings,
        "DEBT": debt,
        "RATIO_SAVINGS_INCOME": ratio_savings_income,
        "RATIO_DEBT_INCOME": ratio_debt_income,
        "RATIO_DEBT_SAVINGS": ratio_debt_savings
    }])

    predict_loan = model.predict(input_data)[0]

    return f"Predicted Safe Loan Amount: ₹{round(predict_loan,2)}"

income = float(input("Enter income: "))
savings = float(input("Enter savings: "))
debt = float(input("Enter debt: "))
loan_duration = int(input("Enter loan duration in months: "))
change_Income = bool(input("Is there a change in income between loan duration? (True/False): "))

predict_safe_loan(income,savings,debt,loan_duration,change_Income)

In [12]:
model = joblib.load("loan_predictor_model.pkl")

def predict_safe_loan(income, savings, debt, loan_duration_months, change_income):
    loan_duration_years = loan_duration_months / 12  

    adjusted_income = income
    ratio_savings_income = savings / adjusted_income if adjusted_income > 0 else 0
    ratio_debt_income = debt / adjusted_income if adjusted_income > 0 else 0
    ratio_debt_savings = debt / savings if savings > 0 else 0

    if change_income:
        new_income = float(input("Enter new annual income: "))
        change_after_months = int(input("Enter months after which income changes: "))
        change_after_years = change_after_months / 12

        adjusted_income = ((income * (loan_duration_years - change_after_years)) + 
                           (new_income * change_after_years)) / loan_duration_years

        ratio_savings_income = savings / adjusted_income if adjusted_income > 0 else 0
        ratio_debt_income = debt / adjusted_income if adjusted_income > 0 else 0
        ratio_debt_savings = debt / savings if savings > 0 else 0

    max_debt_to_income = 3.5  
    max_debt_to_savings = 3

    if ratio_debt_income > max_debt_to_income or ratio_debt_savings > max_debt_to_savings:
        required_repayment = max(debt - (income * 3), 0)
        return f"❌ Not eligible for loan. Must repay at least ₹{round(required_repayment, 2)} first."

    input_data = pd.DataFrame([{
        "INCOME": adjusted_income,
        "SAVINGS": savings,
        "DEBT": debt,
        "R_SAVINGS_INCOME": ratio_savings_income,
        "R_DEBT_INCOME": ratio_debt_income,
        "R_DEBT_SAVINGS": ratio_debt_savings
    }])


    predicted_loan = model.predict(input_data)[0]

    max_safe_loan = min(income*0.4 , savings *0.5)
    final_loan = min(predicted_loan, max_safe_loan)

    return f"Predicted Safe Loan Amount: ₹{round(final_loan, 2)}"

income = float(input("Enter current annual income: "))
savings = float(input("Enter total savings: "))
debt = float(input("Enter total debt: "))
loan_duration_months = int(input("Enter loan duration in months: "))

change_income = input("Will your income change during the loan duration? (yes/no): ").strip().lower() in ['yes', 'true', '1']

result = predict_safe_loan(income, savings, debt, loan_duration_months, change_income)
print("income, savings, debt, loan_duration_months")
print(income, savings, debt, loan_duration_months,)
print(result)

income, savings, debt, loan_duration_months
100000.0 100000.0 250000.0 24
Predicted Safe Loan Amount: ₹40000.0
